## Download a Original Data

* if has not dataset, you must do unlock comment and excute.

In [31]:
# !./init y19

## init

In [32]:
from typing import List
import os

import numpy as np
import pandas as pd

import csv
import shutil

from utils.dfl import dfl_base, dfl_tools

## Set global variable

In [33]:
dir_name = "KEMDy19"
origin_dp = f"{dir_name}"

## Make Functions

In [34]:
def read_csv_info(
    file_path: str=""
) -> pd.DataFrame:
    
    csv_list = []
    with open(file_path) as f:
        r = csv.reader(f)

        for l in r:
            csv_list.append(l)

    return pd.DataFrame(csv_list)


def make_merged_data(
    target_dir_list: List[str],
    file_list: List[str],
    session_name: str
) -> pd.DataFrame:

    assert session_name != None
    
    res: pd.DataFrame = None
        
    for _f in file_list:
        _fds = {}
        
        for tdp in target_dir_list:
            _td = tdp.split("/")
            
            # print(f"{session_name}, {file}, {target_dir[1]}")
            # Directory path form name is as like "KEMDy19/EDA/Session01/Sess01_script01_User001F.csv"
            _fds[_td[1]] = read_csv_info(f"./{_td[0]}/{_td[1]}/{session_name}/Original/{_f}")
            
            if _td[1] == "ECG":
                _fds[_td[1]] = _fds[_td[1]].drop(columns=_fds[_td[1]].columns[0])
            else:
                _fds[_td[1]] = _fds[_td[1]].drop(columns=_fds[_td[1]].columns[1])
            _fds[_td[1]] = _fds[_td[1]].T.reset_index(drop=True).T
        
        # ** for check data
        # for x in target_dir_list:
        #     t = x.split("/")
        #     print(f"{t[1]} : ")
        #     print(file_datas[t[1]].to_string())
        
        # attatching column names
        
        _fds["EDA"] = _fds["EDA"].set_axis(["eda", "timestamp", "sid"], axis=1)
        _fds["ECG"] = _fds["ECG"].set_axis(["ecg", "timestamp", "sid"], axis=1)
        _fds["TEMP"] = _fds["TEMP"].set_axis(["temp", "timestamp", "sid"], axis=1)

        # Merge "TEMP" table to "EDA" 
        _opd = pd.merge(
            _fds["EDA"], _fds["TEMP"],
            left_on='timestamp', right_on='timestamp', how='outer')
        del _opd["sid_x"]
        _opd = _opd.rename(columns={"sid_y": "sid"})

        # Merge "ECG" table to merged table("EDA" and "TEMP")
        _opd = pd.merge(
            _opd, _fds["ECG"],
            left_on='timestamp', right_on='timestamp', how='outer')
        _opd["sid_x"] = _opd["sid_x"].fillna(_opd["sid_y"])
        del _opd["sid_y"]
        _opd = _opd.rename(columns={"sid_x": "sid"})

        # reorder following as "timestamp", "sid", "eda", "temp", "ecg"
        _opd = _opd[["timestamp", "sid", "eda", "temp", "ecg"]]
        # sorting values from "timestamp" column
        _opd = _opd.sort_values("timestamp")
        _opd = _opd.reset_index(drop=True)
        
        if res is None:
            res = _opd
        else:
            res = pd.concat([res, _opd], sort=True)
            # result = result.append(one_part_data_on_session, ignore_index=True)
            
    res = res[["timestamp", "sid", "eda", "temp", "ecg"]]
    res = res.sort_values("timestamp")
    res = res.reset_index(drop=True)
    
    # print(result.to_string())
    
    return res

## Get organized dataset

In [35]:
# make wav directory to root
org_dp = f"org_{dir_name}"

dfl_base.make_dir(".", org_dp)

_wd = dfl_tools.find_dfl_path(f"./{dir_name}", ["wav"], mode="d", only_leaf="True")
_sl = dfl_tools.find_dfl_path(f"{_wd[0]}", [""], mode="d", only_leaf="True", res_all=True)

for l, r in _sl:
    dfl_base.make_dir(f"./{org_dp}", r)
    _ssl = dfl_tools.find_dfl_path(f"{l}", [""], mode="d", only_leaf="True")
    
    _mv_path = f"./{org_dp}/{r}"
    
    for _d in _ssl:
        _fl = dfl_tools.find_dfl_path(_d, [""], mode="f", only_leaf="True")
        for f in _fl:
            shutil.move(f, _mv_path)

src_ap = os.path.join(os.getcwd(), f"{origin_dp}/annotation")
dest_ap = os.path.join(os.getcwd(), f"{org_dp}/annotation")

target_dir_list = ["EDA", "ECG", "TEMP"]

sd_list = []

for _td in target_dir_list:
    _sdl = dfl_tools.find_dfl_path(f"./{origin_dp}/{_td}", ["Session"], mode="d", only_leaf=True, res_all=True)
    sd_list += [x[1] for x in _sdl]
    sd_list = list(set(sd_list))

sd_list = sorted(sd_list)

for _sn in sd_list:
    _tf = []
    
    for _td in target_dir_list:
        _sfl = dfl_tools.find_dfl_path(f"./{origin_dp}/{_td}/{_sn}", [""], mode="f", recur=True, only_leaf=True, res_all=True)
        _tf += [x[1] for x in _sfl]
        
    _tf = list(set(_tf))
    _tf = sorted(_tf)
    
    _smd = make_merged_data([f"{origin_dp}/{x}" for x in target_dir_list], _tf, _sn)
    _smd.to_csv(f"{org_dp}/{_sn}/{_sn}.csv", sep=",", na_rep="NaN")